In [16]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2.extras import DictCursor
from minio import Minio
import os


DETAIL_CRAWL = 'detail_crawl'

conn = psycopg2.connect(user="dpacrawler",
                        password="123456a@",
                        host="127.0.0.1",
                        port="5432",
                        database="dpacrawlconf")


client = Minio(
    endpoint="lakedpaapi-fis-mbf-dplat.apps.xplat.fis.com.vn",
    access_key="I5pnix8qE2mtXlXR",
    secret_key="hzADsWEM8DGIIQBrfjNWNNy4j0OG0cSA",
    secure=False
)

In [17]:
import sys

sys.path.append('../')
from dejavu import Dejavu
config = {
    "database": {
        "host": "127.0.0.1",
        "user": "dpacrawler",
        "password": "123456a@",
        "database": "dpacrawlconf"
    },
    "database_type" : "postgres"
}

djv = Dejavu(config)

In [3]:
def remove_file():
    for dir in os.listdir('../mp3'):
        os.remove(f'../mp3/{dir}')

In [ ]:
from psycopg2.extensions import AsIs
with djv.db.cursor() as cur:
    exps = str(input('Enter condition for id:'))
    query = f"""
        SELECT id,name,path FROM {DETAIL_CRAWL}
        where id %s;
    """
    cur.execute(query,[AsIs(exps)])
    while True:
        remove_file()
        rows = cur.fetchmany(50)
        if not rows:
            break
        for row in rows:
            print(row)

In [13]:
def get_song_info():
    global conn
    global client
    cur = conn.cursor()
    remove_file()
    song_info = {}
    song_revert = {}
    query = f"""
        SELECT id,name,path FROM {DETAIL_CRAWL}
        limit 10
    """
    cur.execute(query)
    while True:
        remove_file()
        rows = cur.fetchmany(50)
        if not rows:
            break
        for row in rows:
            list_path = row[2].split('/')
            bucket_name = list_path[0]
            row_2 = '/'.join(list_path[1:])
            song_info[row[0]] = [row[1],row_2]
            print(bucket_name)
            print(row_2)
            # song_revert[row[1]] = [row[0],row[2]]
            client.fget_object(bucket_name=bucket_name,object_name=row_2,file_path=f'../mp3/{row[0]}.mp3')
        # djv.fingerprint_directory("../mp3", [".mp3"], 4,song_info)
    cur.close()

In [84]:
client.fget_object(bucket_name='youtube',object_name='Dunghoangpham/See Tình - Hoàng Thùy Linh  Dunghoangpham Cover.mp3',file_path=f'../test/442.mp3')

In [104]:
# for x in client.list_objects('youtube',recursive=True):
#     print(x.object_name)

In [ ]:
get_song_info()

In [67]:
from dejavu.logic.recognizer.file_recognizer import FileRecognizer
song = djv.recognize(FileRecognizer, "../test/dalab.mp3")

In [85]:
import abc
from time import time
from typing import Dict, List, Tuple
from time import time
from typing import Dict

import dejavu.logic.decoder as decoder
from dejavu.base_classes.base_recognizer import BaseRecognizer
from dejavu.config.settings import (ALIGN_TIME, FINGERPRINT_TIME, QUERY_TIME,
                                    RESULTS, TOTAL_TIME)
import numpy as np

from dejavu.config.settings import DEFAULT_FS
filename = '../test/442.mp3'
channels, djv.Fs, _ = decoder.read(filename, djv.limit)

In [86]:
fingerprint_times = []
hashes = set()  # to remove possible duplicated fingerprints we built a set.
for channel in channels:
    fingerprints, fingerprint_time = djv.generate_fingerprints(channel, Fs=djv.Fs)
    fingerprint_times.append(fingerprint_time)
    hashes |= set(fingerprints)

matches, dedup_hashes, query_time = djv.find_matches(hashes)

In [87]:
def calculate_time(s,e):
    return (s - e)/DEFAULT_FS * DEFAULT_WINDOW_SIZE * DEFAULT_OVERLAP_RATIO

In [88]:
import multiprocessing
import os
import sys
import traceback
from itertools import groupby
from time import time
import numpy as np
from typing import Dict, List, Tuple

import dejavu.logic.decoder as decoder
from dejavu.base_classes.base_database import get_database
from dejavu.config.settings import (DEFAULT_FS, DEFAULT_OVERLAP_RATIO,
                                    DEFAULT_WINDOW_SIZE, FIELD_FILE_SHA1,
                                    FIELD_TOTAL_HASHES,FIELD_SONG_ID,
                                    FINGERPRINTED_CONFIDENCE,
                                    FINGERPRINTED_HASHES, HASHES_MATCHED,
                                    INPUT_CONFIDENCE, INPUT_HASHES, OFFSET,
                                    OFFSET_SECS, SONG_ID, SONG_NAME, TOPN,TIME_CONSIDER)
queried_hashes = len(hashes)
from dejavu.logic.fingerprint import fingerprint
sorted_matches = sorted(matches, key=lambda m: (m[0], m[1]))
counts = [(*key, len(list(group))) for key, group in groupby(sorted_matches, key=lambda m: (m[0], m[1]))]
songs_matches = sorted(
    [max(list(group), key=lambda g: g[2]) for key, group in groupby(counts, key=lambda count: count[0])],
    key=lambda count: count[2], reverse=True
)

songs_result = []
list_tmp = []
for song_id, offset, fingerprint_match in songs_matches[0:5]:  # consider topn elements in the result
    song = djv.db.get_song_by_id(song_id)

    song_name = song.get(SONG_NAME, None)
    song_hashes = song.get(FIELD_TOTAL_HASHES, None)
    nseconds = round(float(offset) / DEFAULT_FS * DEFAULT_WINDOW_SIZE * DEFAULT_OVERLAP_RATIO, 5)
    hashes_matched = dedup_hashes[song_id]

    list_offset = set()
    for id, count_match,time_match in sorted_matches:
        if id == song_id and offset == count_match:
            list_offset.add(time_match)
    list_offset = list(list_offset)
    list_offset.sort()
    time_similar = 0.0
    for iter in range(0,len(list_offset)):
        if iter == 0:
            continue
        delta = calculate_time(list_offset[iter],list_offset[iter -1])
        if delta <= 1.5:
            time_similar += delta
    # min_time = np.min(list(list_offset))
    # max_time = np.max(list(list_offset))
    # list_tmp.append(list(list_offset))
    # # if(len(list_offset) < 50):
    # #     continue

    accuracy_similar = 100 if time_similar > TIME_CONSIDER else time_similar/TIME_CONSIDER*100

    song = {
        'original_song_id':1,
        'original_song_name':'test.mp3',
        SONG_ID: song_id,
        SONG_NAME: song_name,
        INPUT_HASHES: queried_hashes,
        FINGERPRINTED_HASHES: song_hashes,
        'time_similar': float(time_similar),
        'accuracy_similar':accuracy_similar,
        'fingerprint_match':fingerprint_match,
        HASHES_MATCHED: hashes_matched,
        # Percentage regarding hashes matched vs hashes from the input.
        INPUT_CONFIDENCE: round(hashes_matched / queried_hashes, 2),
        # Percentage regarding hashes matched vs hashes fingerprinted in the db.
        FINGERPRINTED_CONFIDENCE: round(hashes_matched / song_hashes, 2),
        OFFSET: int(offset),
        OFFSET_SECS: nseconds,
        FIELD_FILE_SHA1: song.get(FIELD_FILE_SHA1, None).encode("utf8")
    }

    # djv.db.insert_check_song(list(song.values())[:-1])

    songs_result.append(song)

In [89]:
songs_result

[{'original_song_id': 1,
  'original_song_name': 'test.mp3',
  'song_id': 46,
  'song_name': 'Hoàng Thuỳ Linh - See Tình  Official Music Video.mp3',
  'input_total_hashes': 62429,
  'fingerprinted_hashes_in_db': 97821,
  'time_similar': 7.801904761904762,
  'accuracy_similar': 26.006349206349206,
  'fingerprint_match': 24,
  'hashes_matched_in_input': 6704,
  'input_confidence': 0.11,
  'fingerprinted_confidence': 0.07,
  'offset': 715,
  'offset_seconds': 33.20454,
  'file_sha1': b'5D32843F92D7EDC3ACFAD5CC1C31D506E6A4B6DF'},
 {'original_song_id': 1,
  'original_song_name': 'test.mp3',
  'song_id': 44,
  'song_name': 'Hoàng Thuỳ Linh - See Tình  Dance Performance.mp3',
  'input_total_hashes': 62429,
  'fingerprinted_hashes_in_db': 92253,
  'time_similar': 2.2291156462585033,
  'accuracy_similar': 7.430385487528344,
  'fingerprint_match': 17,
  'hashes_matched_in_input': 6307,
  'input_confidence': 0.1,
  'fingerprinted_confidence': 0.07,
  'offset': 52,
  'offset_seconds': 2.41488,
  '

In [75]:
list_tmp[0].sort()
set(list_tmp[0])

{2102, 2143, 2156, 2184, 3408, 3435, 3462, 3489, 3516}

In [83]:
(2143 - 2102)/DEFAULT_FS * DEFAULT_WINDOW_SIZE * DEFAULT_OVERLAP_RATIO

1.9040362811791383

In [ ]:
(2143 - 2102)/DEFAULT_FS * DEFAULT_WINDOW_SIZE * DEFAULT_OVERLAP_RATIO